In [23]:
!pip install --pre deepchem[tensorflow]

In [24]:
import deepchem as dc
import numpy as np
import timeit
import pandas as pd

from sklearn.utils._testing import ignore_warnings
from sklearn.exceptions import ConvergenceWarning
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF,RationalQuadratic, Matern, WhiteKernel
from sklearn.feature_selection import RFE
from sklearn.tree import DecisionTreeRegressor
from numpy import average

In [25]:
import math

In [26]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [54]:


#function to get the names of the descriptors based on the support vector
def Get_Selected_features(support,descriptorlist):
    descriptors = []
    for idx, bool in enumerate(support):
        if bool == True:
            descriptors.append(descriptorlist[idx])
    return descriptors


@ignore_warnings(category=ConvergenceWarning)
def main(name):

    # create a list that stores everything that needs to written to the result file
    text = []

    #generate the list of RDKit descriptors
    from rdkit.Chem import Descriptors
    descriptors = Descriptors._descList

    #make a list with the names of descriptors available on RDKit
    descriptorlist = []
    for item in descriptors:
        descriptorlist.append(item[0])


    #this is done since RDKit utility in deepchem calculates the descriptors in alphabetical order,
    # this way we can later extract the used descriptors
    descriptorlist = sorted(descriptorlist)

    # and add the simulation temp
    descriptorlist.append("Tsim")

    print("Results of:", name)
    text.append(["".join(map(str,["Results of: ", name]))])

    task = "density"
    file = '/content/drive/MyDrive/Colab Notebooks/Master_Thesis/Predicting_T_dependency/csv_files/Combined_dens_Dataset_With'+ name+ '.csv'

    print("\nloading the data...")
    loader = dc.data.CSVLoader([task], feature_field="smiles", featurizer=dc.feat.RDKitDescriptors())
    Data = loader.create_dataset(file)
    print("Data loaded")

    #some RDKit descriptors return nan, make these 0
    X = np.nan_to_num(Data.X, copy=True, nan=0.0, posinf=0)
    print("original shape:",X.shape)
    # now load the additional features
    df = pd.read_csv(file)
    Ts = df["temp"].to_numpy()
    # combine the RDKit descriptors with the simulation temperature
    input=np.column_stack((X,Ts))
    print("With TS added:",input.shape)

    #generate the dataset
    Dataset = dc.data.DiskDataset.from_numpy(X=input, y=Data.y, w=Data.w, ids=Data.ids, tasks = [task])


    text.append(["".join(map(str,["\n\npredicting ",task,"..."]))])
    print("\npredicting",task,"...")

    numbers = [6,11,16,21,31,41,51,61,71,101,151,211]

    for n_of_feats in numbers:
        text.append(["".join(map(str,["\n\nnumber of feats: ", n_of_feats]))])
        print("\n\nnumber of feats:",n_of_feats)

        #select the features using a randomforest (decisionTreeRegressor)
        selector = RFE(estimator=DecisionTreeRegressor(), n_features_to_select=n_of_feats, step = 150)
        X_Selected = selector.fit_transform(Dataset.X, Dataset.y.ravel()) #y is a colum vector but it needs a 1d array -> ravel fixes this
        RFEDataset = dc.data.DiskDataset.from_numpy(X=X_Selected, y=Dataset.y, w=Dataset.w, ids=Dataset.ids, tasks = [task])

        #find which descriptors are the most important
        selected = selector.support_ #this returns an array with true and false, True are the selected features
        desc = Get_Selected_features(selected,descriptorlist)
        text.append(desc)

        #initiate lists to keep the results
        train_r2scores = []
        valid_r2scores = []
        test_r2scores = []
        RMSE_scores = []
        start = timeit.default_timer()


        #initiate a loop of 5 times in order to get a decent estimation on the models performance
        for i in range(5):

            #split the dataset using the random splitter
            splitter = dc.splits.RandomSplitter()
            train_dataset, valid_dataset,  test_dataset = splitter.train_valid_test_split(RFEDataset, frac_train = 0.8, frac_valid = 0.1, frac_test = 0.1)


            # create the GPR model & fit the model
            kernel = 1 * RationalQuadratic() + WhiteKernel()
            model = dc.models.SklearnModel(GaussianProcessRegressor(kernel=kernel, n_restarts_optimizer=5))

            #fit the model
            model.fit(train_dataset)


            #predict the test set
            predicted_test = model.predict(test_dataset)

            #calculate r2 scores
            metric = dc.metrics.Metric(dc.metrics.pearson_r2_score)
            train_r2score = model.evaluate(train_dataset, metric)
            valid_r2score = model.evaluate(valid_dataset, metric)
            test_r2score= model.evaluate(test_dataset, metric)

            #make then useable
            testr2=list(test_r2score.values())[0]
            validr2=list(valid_r2score.values())[0]
            trainr2=list(train_r2score.values())[0]

            #calculate RMSE score
            from sklearn.metrics import mean_squared_error
            MSE_score = mean_squared_error(test_dataset.y,predicted_test)
            RMSE_score = math.sqrt(MSE_score)

            #add them to the list:
            train_r2scores.append(trainr2)
            valid_r2scores.append(validr2)
            test_r2scores.append(testr2)
            RMSE_scores.append(RMSE_score)

        stop = timeit.default_timer()

        #average the results and print to screen
        text.append([''.join(map(str,["average training r2-score: ",round(np.mean(train_r2scores),4)] ))])
        text.append([''.join(map(str,["average valid r2-score: ",round(np.mean(valid_r2scores),4)] ))])
        text.append([''.join(map(str,["average test r2-score: ",round(np.mean(test_r2scores),4 )] ))])
        text.append([''.join(map(str,["average test RMSE-score: ",round(np.mean(RMSE_scores),4)] ))])
        text.append([''.join(map(str,["Time: ",stop-start] ))])



    # now write everytinh to a txt file
    result_file = "/content/drive/MyDrive/Colab Notebooks/Master_Thesis/Predicting_T_dependency/Results_" + name+ ".txt"

    with open(result_file, 'w') as f:
      # Use a for loop to write each line of data to the file
      for line in text:
        for word in line:
          f.write(str(word))
          f.write('\n')


In [55]:
# change this
main("State1")

Results of: State1

loading the data...
Data loaded
original shape: (693, 210)
With TS added: (693, 211)

predicting density ...


number of feats: 6


number of feats: 11


number of feats: 16


number of feats: 21


number of feats: 31


number of feats: 41


number of feats: 51


number of feats: 61


number of feats: 71


number of feats: 101


number of feats: 151


number of feats: 211
